In [1]:
# ---- Import libraries & load data ----
import pandas as pd
from keplergl import KeplerGl
from pathlib import Path
from IPython.display import IFrame
import json

PROC_DIR   = Path(r'C:\Users\moein\anaconda3\citi-bike-2022-weather\data\processed')
REPORT_DIR = Path(r"C:\Users\moein\anaconda3\citi-bike-2022-weather\data\reports\map")
REPORT_DIR.mkdir(parents=True, exist_ok=True)

daily_path = PROC_DIR / "citibike_2022_daily_with_weather.csv"
trips_path = PROC_DIR / "citibike_2022_trips_with_weather.csv"

daily = pd.read_csv(daily_path, parse_dates=["date"])
trips = pd.read_csv(trips_path, parse_dates=["started_at", "ended_at"])
daily.head(), daily.shape

C:\Users\moein\anaconda3\envs\citi_2022\lib\site-packages\keplergl\keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


(        date  rides  member_share  avg_temp_c
 0 2022-01-01    592      0.543919        11.6
 1 2022-01-02   1248      0.584936        11.4
 2 2022-01-03    832      0.772837         1.4
 3 2022-01-04    934      0.776231        -2.7
 4 2022-01-05    914      0.750547         3.2,
 (365, 4))

In [3]:
trips.head(), trips.shape

(            ride_id  rideable_type          started_at            ended_at  \
 0  CA5837152804D4B5  electric_bike 2022-01-26 18:50:39 2022-01-26 18:51:53   
 1  BA06A5E45B6601D2   classic_bike 2022-01-28 13:14:07 2022-01-28 13:20:23   
 2  7B6827D7B9508D93   classic_bike 2022-01-10 19:55:13 2022-01-10 20:00:37   
 3  6E5864EA6FCEC90D  electric_bike 2022-01-26 07:54:57 2022-01-26 07:55:22   
 4  E24954255BBDE32D  electric_bike 2022-01-13 18:44:46 2022-01-13 18:45:43   
 
      start_station_name start_station_id      end_station_name end_station_id  \
 0  12 St & Sinatra Dr N            HB201  12 St & Sinatra Dr N          HB201   
 1      Essex Light Rail            JC038      Essex Light Rail          JC038   
 2      Essex Light Rail            JC038      Essex Light Rail          JC038   
 3  12 St & Sinatra Dr N            HB201  12 St & Sinatra Dr N          HB201   
 4  12 St & Sinatra Dr N            HB201  12 St & Sinatra Dr N          HB201   
 
    start_lat  start_lng    en

In [4]:
# ---- Aggregate trips by station pairs ----

# 1) Helper flag to count rows
trips["value"] = 1

# 2) Select minimal columns needed for flows
cols = [
    "start_station_name", "start_station_id", "start_lat", "start_lng",
    "end_station_name", "end_station_id", "end_lat", "end_lng",
    "value"]
missing = [c for c in cols if c not in trips.columns]
assert not missing, f"Missing Columns: {missing}"

# 3) Drop rows without coordinates (keplegl needs lng/lat)
_trips = trips.dropna(subset=["start_lat", "start_lng", "end_lat", "end_lng"]).copy()

# 4) Aggregate counts per station pair
trip_flows = (
    _trips
    .groupby([
        "start_station_name", "start_station_id", "start_lat", "start_lng",
        "end_station_name", "end_station_id", "end_lat", "end_lng"
    ], as_index=False)["value"]
    .count()
    .rename(columns={"value": "trips"})
)

trip_flows.head(), trip_flows.shape

(      start_station_name start_station_id  start_lat  start_lng  \
 0  11 St & Washington St            HB502  40.747251 -74.027879   
 1  11 St & Washington St            HB502  40.747251 -74.027879   
 2  11 St & Washington St            HB502  40.749625 -74.027652   
 3  11 St & Washington St            HB502  40.749716 -74.027232   
 4  11 St & Washington St            HB502  40.749737 -74.027317   
 
                            end_station_name end_station_id    end_lat  \
 0                     11 St & Washington St          HB502  40.749985   
 1                      12 St & Sinatra Dr N          HB201  40.750604   
 2  Hoboken Terminal - Hudson St & Hudson Pl          HB101  40.735938   
 3          Church Sq Park - 5 St & Park Ave          HB601  40.742659   
 4                        Willow Ave & 12 St          HB505  40.751867   
 
      end_lng  trips  
 0 -74.027150      1  
 1 -74.024020      1  
 2 -74.030305      1  
 3 -74.032233      1  
 4 -74.030377      1  ,
 (120

In [5]:
trip_flows["trips"].sum(), len(_trips)

(892281, 893515)

In [6]:
# ---- Initiate a Kepler.Gl map ----
m = KeplerGl(height=720, data={"Trip Flows": trip_flows})
m.save_to_html(file_name="citibike_trip_flows_2022.html",
               read_only=False, config=m.config)
IFrame("citibike_trip_flows_2022.html", width="100%", height=720)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to citibike_trip_flows_2022.html!


## Map Styling Rationale

- I set **start** and **end** stations as **neutral point layers** so they don’t compete with the flow layer visually.  
- I used an **Arc layer** to depict **start → end** flows and mapped **color** to `trips`. A darker/more intense color indicates more traffic; this makes the busiest connections pop at a glance.  
- I kept **radius/width** modest and reduced **opacity** to avoid overplotting while preserving density cues.  
- I deleted redundant autogenerated layers to keep the visual narrative tight: **stations (context)** + **flows (insight)**.

## Findings from Filtering (NYC 2022 Trips)

- Increasing the **`trips`** threshold quickly isolates a core network of high-volume flows centered in **Manhattan**, especially around **Midtown**, **Downtown**, and key river crossings.  
- **Tourist and commuter hotspots** (e.g., Central Park periphery, Times Square, Wall Street, and Grand Central area) sustain the densest connections.  
- The map shows strong clustering where **employment centers**, **tourism landmarks**, and **dense station coverage** overlap.  
- **Impression:** demand is heavily concentrated along business and tourist corridors. This suggests that **capacity planning and bike redistribution** should prioritize these high-traffic zones to reduce shortages and improve service reliability.


In [7]:
# capture UI config so the styling is reproducible
config = m.config

# export interactive HTML (opens in any browser)
m.save_to_html(
    file_name="citibike_trip_flows_2022.html",
    read_only=False,
    config=config
)

# also save the config to JSON for versioning
import json
with open("kepler_config_trip_flows.json", "w") as f:
    json.dump(config, f)

Map saved to citibike_trip_flows_2022.html!


In [8]:
html_path = REPORT_DIR / "citibike_trip_flows_2022.html"
json_path = REPORT_DIR / "kepler_config_trip_flows.json"

m.save_to_html(file_name=str(html_path), read_only=False, config=config)

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f"Saved HTML to: {html_path}")
print(f"Saved config to: {json_path}")

Map saved to C:\Users\moein\anaconda3\citi-bike-2022-weather\data\reports\map\citibike_trip_flows_2022.html!
Saved HTML to: C:\Users\moein\anaconda3\citi-bike-2022-weather\data\reports\map\citibike_trip_flows_2022.html
Saved config to: C:\Users\moein\anaconda3\citi-bike-2022-weather\data\reports\map\kepler_config_trip_flows.json
